## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import models, datasets, transforms
from sklearn.model_selection import StratifiedKFold
import numpy as np
from utils import train_model, evaluate_model
import random

## Set seed for reproducibility

In [2]:
def set_seed(seed):
    """
    Set the seed for reproducibility.

    Args:
        seed (int): Seed value to set for random number generation.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Set seed to be 42
set_seed(42)

## Inception-v1

In [3]:
# Use CUDA if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the InceptionV1 model to use
def create_inception_v1_model(num_classes):
    # Load the GoogleNet model (aka. InceptionV1)
    model = models.googlenet(pretrained=False, aux_logits=False)
    # Replace the classifier layer
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)

## Helper Functions

In [4]:
import itertools
import torch
import numpy as np

def load_dataset(dataset_folder):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.436, 0.385, 0.344], std=[0.296, 0.269, 0.261])
    ])
    
    dataset = datasets.ImageFolder(root=dataset_folder, transform=transform)
    return dataset
    
def grid_search_tuning(dataset, num_classes, learning_rate, batch_sizes, num_epochs=5):
    best_accuracy = 0
    best_f1 = 0
    best_params = {}

    for lr, bs in itertools.product(learning_rates, batch_sizes):
        print(f"Testing LR: {lr}, BS: {bs}")

        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        targets = np.array([dataset.targets[i] for i in range(len(dataset))])
        fold_accuracies = []
        fold_f1_scores = []

        for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(dataset)), targets)):
            print(f'Fold {fold + 1}')
            
            train_sampler = Subset(dataset, train_idx)
            val_sampler = Subset(dataset, val_idx)
            
            train_dataloader = DataLoader(train_sampler, batch_size=bs, shuffle=True)
            val_dataloader = DataLoader(val_sampler, batch_size=bs, shuffle=False)
            
            model = create_inception_v1_model(num_classes)
            trained_model = train_model(model, train_dataloader, val_dataloader, num_epochs=num_epochs, lr=lr)
            
            fold_accuracy, fold_f1 = evaluate_model(trained_model, val_dataloader)
            fold_accuracies.append(fold_accuracy)
            fold_f1_scores.append(fold_f1)
            print(f'Fold {fold + 1} Accuracy: {fold_accuracy:.4f}, F1 Score: {fold_f1:.4f}')
        
        mean_accuracy = np.mean(fold_accuracies)
        mean_f1 = np.mean(fold_f1_scores)
        print(f'Mean Accuracy for LR: {lr}, BS: {bs}: {mean_accuracy:.4f}')
        print(f'Mean F1 Score for LR: {lr}, BS: {bs}: {mean_f1:.4f}')

        if mean_accuracy > best_accuracy:
            best_accuracy = mean_accuracy
            best_f1 = mean_f1
            best_params = {'learning_rate': lr, 'batch_size': bs}
    
    print(f'Best Parameters: {best_params}')
    print(f'Best Accuracy: {best_accuracy:.4f}')
    print(f'Best F1 Score: {best_f1:.4f}')

## Start the tuning

In [5]:
# Set up device and dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
dataset_folder = '../data/lfw'
dataset = load_dataset(dataset_folder)
num_classes = len(dataset.classes)

In [5]:
# Define ranges for hyperparameters
learning_rates = [0.01, 0.1]
batch_sizes = [32, 64, 16]

# Perform grid search tuning
grid_search_tuning(dataset, num_classes, learning_rates, batch_sizes, num_epochs=50)

Testing LR: 0.01, BS: 32
Fold 1


/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torchvision/models/googlenet.py:46: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1/50, Train Loss: 3.3939, Val Loss: 12.1051
Epoch 2/50, Train Loss: 2.5333, Val Loss: 2.8046
Epoch 3/50, Train Loss: 2.3743, Val Loss: 2.3441
Epoch 4/50, Train Loss: 2.2970, Val Loss: 3.8521
Epoch 5/50, Train Loss: 2.2201, Val Loss: 2.2419
Epoch 6/50, Train Loss: 2.1553, Val Loss: 2.4059
Epoch 7/50, Train Loss: 2.0238, Val Loss: 2.5497
Epoch 8/50, Train Loss: 2.0195, Val Loss: 2.2886
Epoch 9/50, Train Loss: 1.9036, Val Loss: 2.1547
Epoch 10/50, Train Loss: 1.6860, Val Loss: 3.4396
Epoch 11/50, Train Loss: 1.7111, Val Loss: 1.9153
Epoch 12/50, Train Loss: 1.6016, Val Loss: 3.7078
Epoch 13/50, Train Loss: 1.4494, Val Loss: 3.9820
Epoch 14/50, Train Loss: 1.4742, Val Loss: 2.1388
Epoch 15/50, Train Loss: 1.2863, Val Loss: 1.9629
Epoch 16/50, Train Loss: 1.1877, Val Loss: 1.7277
Epoch 17/50, Train Loss: 1.1285, Val Loss: 2.1784
Epoch 18/50, Train Loss: 1.0475, Val Loss: 2.1043
Epoch 19/50, Train Loss: 1.0080, Val Loss: 1.8966
Epoch 20/50, Train Loss: 0.9314, Val Loss: 1.7777
Epoch 21

/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torchvision/models/googlenet.py:46: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1/50, Train Loss: 2.8705, Val Loss: 3652.7016
Epoch 2/50, Train Loss: 2.4065, Val Loss: 11.4984
Epoch 3/50, Train Loss: 2.3716, Val Loss: 6.9683
Epoch 4/50, Train Loss: 2.3081, Val Loss: 2.3147
Epoch 5/50, Train Loss: 2.3103, Val Loss: 2.3430
Epoch 6/50, Train Loss: 2.2052, Val Loss: 2.1749
Epoch 7/50, Train Loss: 2.2004, Val Loss: 2.3221
Epoch 8/50, Train Loss: 2.0927, Val Loss: 1.9697
Epoch 9/50, Train Loss: 2.0316, Val Loss: 2.2604
Epoch 10/50, Train Loss: 1.8929, Val Loss: 2.0581
Epoch 11/50, Train Loss: 1.8744, Val Loss: 2.0009
Epoch 12/50, Train Loss: 1.7650, Val Loss: 1.8882
Epoch 13/50, Train Loss: 1.7441, Val Loss: 4.2475
Epoch 14/50, Train Loss: 1.7186, Val Loss: 1.9352
Epoch 15/50, Train Loss: 1.6433, Val Loss: 2.3424
Epoch 16/50, Train Loss: 1.6925, Val Loss: 3.0989
Epoch 17/50, Train Loss: 1.4991, Val Loss: 1.7309
Epoch 18/50, Train Loss: 1.2372, Val Loss: 1.4945
Epoch 19/50, Train Loss: 1.2016, Val Loss: 2.9157
Epoch 20/50, Train Loss: 1.2071, Val Loss: 4.7595
Epoch

...

Epoch 1/50, Train Loss: 4.7494, Val Loss: 108.2412
Epoch 2/50, Train Loss: 2.3128, Val Loss: 2.2980
Epoch 3/50, Train Loss: 2.2730, Val Loss: 2.3871
Epoch 4/50, Train Loss: 2.2458, Val Loss: 2.2147
Epoch 5/50, Train Loss: 2.2625, Val Loss: 2.2563
Epoch 6/50, Train Loss: 2.2476, Val Loss: 2.2685
Epoch 7/50, Train Loss: 2.2522, Val Loss: 2.1931
Epoch 8/50, Train Loss: 2.2284, Val Loss: 2.2254
Epoch 9/50, Train Loss: 2.1866, Val Loss: 2.2897
Epoch 10/50, Train Loss: 2.2207, Val Loss: 2.9382
Epoch 11/50, Train Loss: 2.2199, Val Loss: 2.3204
Epoch 12/50, Train Loss: 2.1530, Val Loss: 2.2316
Early stopping triggered after 12 epochs
Training complete
Accuracy: 0.2329
F1 Score: 0.0860
Fold 1 Accuracy: 0.2329, F1 Score: 0.0860
Fold 2
Epoch 1/50, Train Loss: 5.5919, Val Loss: 25.8664
Epoch 2/50, Train Loss: 2.3352, Val Loss: 2.5354
Epoch 3/50, Train Loss: 2.2984, Val Loss: 2.2713
Epoch 4/50, Train Loss: 2.2973, Val Loss: 2.2582
Epoch 5/50, Train Loss: 2.2861, Val Loss: 2.2684
Epoch 6/50, Train L

/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torchvision/models/googlenet.py:46: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1/50, Train Loss: 8.0541, Val Loss: 125662.3783
Epoch 2/50, Train Loss: 2.4925, Val Loss: 57.6846
Epoch 3/50, Train Loss: 2.3079, Val Loss: 3.1304
Epoch 4/50, Train Loss: 2.3008, Val Loss: 2.3281
Epoch 5/50, Train Loss: 2.2740, Val Loss: 2.3255
Epoch 6/50, Train Loss: 2.2680, Val Loss: 2.2515
Epoch 7/50, Train Loss: 2.2455, Val Loss: 2.3344
Epoch 8/50, Train Loss: 2.2220, Val Loss: 2.3691
Epoch 9/50, Train Loss: 2.1554, Val Loss: 2.1940
Epoch 10/50, Train Loss: 2.1414, Val Loss: 2.2774
Epoch 11/50, Train Loss: 2.0606, Val Loss: 2.2428
Epoch 12/50, Train Loss: 2.0670, Val Loss: 2.1730
Epoch 13/50, Train Loss: 2.0036, Val Loss: 2.1090
Epoch 14/50, Train Loss: 1.9622, Val Loss: 2.2167
Epoch 15/50, Train Loss: 1.9738, Val Loss: 2.2988
Epoch 16/50, Train Loss: 2.1719, Val Loss: 2.3011
Epoch 17/50, Train Loss: 2.0083, Val Loss: 2.0450
Epoch 18/50, Train Loss: 1.8767, Val Loss: 2.4308
Epoch 19/50, Train Loss: 1.9315, Val Loss: 7.5358
Epoch 20/50, Train Loss: 1.8985, Val Loss: 2.6234
Epo

/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torchvision/models/googlenet.py:46: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1/50, Train Loss: 3.9348, Val Loss: 2.4411
Epoch 2/50, Train Loss: 2.3199, Val Loss: 2.3823
Epoch 3/50, Train Loss: 2.2965, Val Loss: 2.3294
Epoch 4/50, Train Loss: 2.2954, Val Loss: 2.3103
Epoch 5/50, Train Loss: 2.2482, Val Loss: 2.2380
Epoch 6/50, Train Loss: 2.3137, Val Loss: 2.2368
Epoch 7/50, Train Loss: 2.2747, Val Loss: 2.2415
Epoch 8/50, Train Loss: 2.2309, Val Loss: 2.2542
Epoch 9/50, Train Loss: 2.2332, Val Loss: 2.2540
Epoch 10/50, Train Loss: 2.2192, Val Loss: 2.2367
Epoch 11/50, Train Loss: 2.2667, Val Loss: 2.3315
Epoch 12/50, Train Loss: 2.1999, Val Loss: 2.2513
Epoch 13/50, Train Loss: 2.2034, Val Loss: 2.4137
Epoch 14/50, Train Loss: 2.1910, Val Loss: 2.1387
Epoch 15/50, Train Loss: 2.1883, Val Loss: 2.1952
Epoch 16/50, Train Loss: 2.1758, Val Loss: 2.6804
Epoch 17/50, Train Loss: 2.1509, Val Loss: 2.0745
Epoch 18/50, Train Loss: 2.1759, Val Loss: 2.3091
Epoch 19/50, Train Loss: 2.1752, Val Loss: 2.4249
Epoch 20/50, Train Loss: 2.1666, Val Loss: 2.1677
Epoch 21/

# Refined Tuning
Let's try with an even smaller batch size.

In [6]:
# Define ranges for hyperparameters
learning_rates = [0.01]
batch_sizes = [4, 8, 16]

# Perform grid search tuning
grid_search_tuning(dataset, num_classes, learning_rates, batch_sizes, num_epochs=50)

Testing LR: 0.01, BS: 4
Fold 1


/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torchvision/models/googlenet.py:46: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1/50, Train Loss: 2.5710, Val Loss: 2.2271
Epoch 2/50, Train Loss: 2.2889, Val Loss: 2.2165
Epoch 3/50, Train Loss: 2.3060, Val Loss: 2.1523
Epoch 4/50, Train Loss: 2.2879, Val Loss: 2.1978
Epoch 5/50, Train Loss: 2.2796, Val Loss: 2.0712
Epoch 6/50, Train Loss: 2.2576, Val Loss: 2.2496
Epoch 7/50, Train Loss: 2.2698, Val Loss: 2.1862
Epoch 8/50, Train Loss: 2.2318, Val Loss: 2.1705
Epoch 9/50, Train Loss: 2.2661, Val Loss: 2.7896
Epoch 10/50, Train Loss: 2.2708, Val Loss: 2.0847
Early stopping triggered after 10 epochs
Training complete
Accuracy: 0.3562
F1 Score: 0.0916
Fold 1 Accuracy: 0.3562, F1 Score: 0.0916
Fold 2
Epoch 1/50, Train Loss: 2.5941, Val Loss: 2.6924
Epoch 2/50, Train Loss: 2.3057, Val Loss: 2.1912
Epoch 3/50, Train Loss: 2.2864, Val Loss: 2.4583
Epoch 4/50, Train Loss: 2.2920, Val Loss: 2.2313
Epoch 5/50, Train Loss: 2.2952, Val Loss: 2.1751
Epoch 6/50, Train Loss: 2.2877, Val Loss: 2.2612
Epoch 7/50, Train Loss: 2.2979, Val Loss: 2.1253
Epoch 8/50, Train Loss: 

/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torchvision/models/googlenet.py:46: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1/50, Train Loss: 2.7078, Val Loss: 2.7337
Epoch 2/50, Train Loss: 2.3537, Val Loss: 2.2373
Epoch 3/50, Train Loss: 2.3267, Val Loss: 2.2427
Epoch 4/50, Train Loss: 2.2984, Val Loss: 3.3230
Epoch 5/50, Train Loss: 2.3031, Val Loss: 2.2283
Epoch 6/50, Train Loss: 2.2698, Val Loss: 2.2667
Epoch 7/50, Train Loss: 2.2312, Val Loss: 2.2808
Epoch 8/50, Train Loss: 2.2232, Val Loss: 2.1767
Epoch 9/50, Train Loss: 2.2461, Val Loss: 2.2359
Epoch 10/50, Train Loss: 2.1483, Val Loss: 2.2147
Epoch 11/50, Train Loss: 2.1531, Val Loss: 2.0626
Epoch 12/50, Train Loss: 2.0647, Val Loss: 2.1460
Epoch 13/50, Train Loss: 2.0653, Val Loss: 2.0470
Epoch 14/50, Train Loss: 1.9372, Val Loss: 2.0207
Epoch 15/50, Train Loss: 1.9211, Val Loss: 1.9732
Epoch 16/50, Train Loss: 1.8596, Val Loss: 1.8903
Epoch 17/50, Train Loss: 1.7784, Val Loss: 1.8782
Epoch 18/50, Train Loss: 1.7532, Val Loss: 1.9894
Epoch 19/50, Train Loss: 1.6915, Val Loss: 2.0765
Epoch 20/50, Train Loss: 1.6745, Val Loss: 2.0058
Epoch 21/

/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torchvision/models/googlenet.py:46: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
/home/sadat/miniconda3/envs/cs721/lib/python3.8/site-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch 1/50, Train Loss: 2.8166, Val Loss: 2.3131
Epoch 2/50, Train Loss: 2.4073, Val Loss: 2.6916
Epoch 3/50, Train Loss: 2.3390, Val Loss: 2.3478
Epoch 4/50, Train Loss: 2.2950, Val Loss: 2.1831
Epoch 5/50, Train Loss: 2.2312, Val Loss: 2.1745
Epoch 6/50, Train Loss: 2.2399, Val Loss: 2.5195
Epoch 7/50, Train Loss: 2.1388, Val Loss: 2.2111
Epoch 8/50, Train Loss: 2.0419, Val Loss: 2.0964
Epoch 9/50, Train Loss: 1.9759, Val Loss: 1.9341
Epoch 10/50, Train Loss: 1.8471, Val Loss: 2.3066
Epoch 11/50, Train Loss: 1.8199, Val Loss: 1.9249
Epoch 12/50, Train Loss: 1.7289, Val Loss: 1.7332
Epoch 13/50, Train Loss: 1.7397, Val Loss: 2.1115
Epoch 14/50, Train Loss: 1.6866, Val Loss: 1.8194
Epoch 15/50, Train Loss: 1.6737, Val Loss: 1.6831
Epoch 16/50, Train Loss: 1.6139, Val Loss: 1.7189
Epoch 17/50, Train Loss: 1.5032, Val Loss: 1.7958
Epoch 18/50, Train Loss: 1.5028, Val Loss: 1.5740
Epoch 19/50, Train Loss: 1.4027, Val Loss: 1.6313
Epoch 20/50, Train Loss: 1.3937, Val Loss: 1.9278
Epoch 21/

## Results

```
Best Parameters: {'learning_rate': 0.01, 'batch_size': 16}
```